In [1]:
import os
import numpy as np
from tensorflow import keras
import tensorflow as tf
import librosa
from matplotlib import pyplot
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
import sys

def get_metadata(file_name):
    if file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '01':
        emo = 'neutral'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '02':
        emo = 'calm'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '03':
        emo = 'happy'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '04':
        emo = 'sad'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '05':
        emo = 'angry'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '06':
        emo = 'fearful'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '07':
        emo = 'disgust'
    elif file_name[-len('03-01-06-02-02-01-07.wav'):][6:8] == '08':
        emo = 'surprised'
        
    if int(file_name[-len('03-01-06-02-02-01-07.wav'):][18:20]) % 2 == 0:  # Gender
        gen = 'female'
    else:
        gen = 'male'
        # 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 
        # 05 = angry, 06 = fearful, 07 = disgust, 
        # 08 = surprised
    return emo, gen


fn = []
for dirname, _, filenames in os.walk('/mount/Project/Project Files/Data/Audio/Audio Speech Sentiment/'):
    for filename in filenames:
        fn.append(os.path.join(dirname, filename))
        

2023-07-23 22:07:07.987116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 22:07:25.992837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import librosa
import matplotlib.pyplot as plt

In [3]:
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from scipy import signal
import numpy as np
import librosa
import librosa.display
import wave
import random as rn
import tensorflow as tf
from keras.utils import to_categorical
import seaborn as sns
%matplotlib inline

In [4]:
def get_feature(file):
    audio, sample_rate = librosa.load(file, 
                                      # res_type='kaiser_fast'
                                     ) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [5]:
# Extract MFCC features
mfcc_features = get_feature(fn[0])

len(mfcc_features)

40

In [6]:
type(mfcc_features)

numpy.ndarray

In [7]:
mfcc_features.shape

(40,)

In [9]:
# mfcc_features

In [10]:
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [11]:
from tqdm import tqdm

# Initialize empty lists
male_features = []
male_labels = []
# female_features = []
# female_labels = []

# Use tqdm to track progress
for curr_file in tqdm(fn, desc="Processing files", 
                      unit="file"):
    curr_feat = get_feature(curr_file)
    curr_meta = get_metadata(curr_file)
    if curr_meta[1] == 'male':
        male_features.append(curr_feat)
        male_labels.append(curr_meta[0])
    # else:
    #     # female_features.append(curr_feat)
    #     # female_labels.append(curr_meta[0])
    #     continue
    
# Define the list of possible values
possible_values = list(set(male_labels))

# Your list of strings to be encoded
# input_strings = ['happy', 'sad', 'angry', 'calm']

# Create a mapping from each value to its index
value_to_index = {value: index for index, value in enumerate(possible_values)}

# Map each string to its corresponding index
encoded_indices = [value_to_index[string] for string in male_labels]

# Perform one-hot encoding
one_hot_matrix = tf.one_hot(indices=encoded_indices, depth=len(possible_values))

# Convert the TensorFlow tensor to a NumPy array (optional)
male_labels_en = one_hot_matrix.numpy()

Processing files: 100%|██████████████████████████████| 1440/1440 [01:48<00:00, 13.29file/s]
2023-07-24 14:06:09.004135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-24 14:06:09.157800: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
male_features = np.array(male_features)

In [14]:
from sklearn.model_selection import train_test_split

male_features_train, male_features_test, male_labels_train_en, male_labels_test_en = train_test_split(
    male_features, male_labels_en, test_size=0.33, random_state=42)

In [17]:
# Convert labels to one-hot encoded vectors
num_classes = len(male_labels_test_en[0])

In [23]:
X_train = male_features_train
X_test = male_features_test
y_train = male_labels_train_en
y_test = male_labels_test_en

In [24]:
# Reshape the features for CNN input (assuming each MFCC feature# Reshape the features for CNN input (assuming each MFCC feature is a row)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

In [28]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(X_train.shape[1], 1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model on the test data
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

ValueError: Exception encountered when calling layer "conv2d_4" (type Conv2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node conv2d_4/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_4/Conv2D/ReadVariableOp)' with input shapes: [?,40,1,1], [2,2,1,32].

Call arguments received by layer "conv2d_4" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 40, 1, 1), dtype=float32)